In [ ]:
import pandas as pd
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv1D, Flatten, Dense, LSTM, Bidirectional
from tensorflow.keras.losses import MeanSquaredError as lMSE
from tensorflow.keras.metrics import CosineSimilarity as mCosS
from tensorflow.keras.metrics import MeanSquaredError as mMSE
from thesispack.models import BaseNeuralNetwork, MetricBase, LossBase
import tensorflow as tf
from thesispack.methods import history_figure
# from bayes_opt import BayesianOptimization
# from functools import partial

In [ ]:
df=pd.read_csv(
    StringIO(
        str(
            np.load("../data/Data.zip")['RTT_data.csv'],
            'utf-8'
    )))

In [ ]:
for k in df.keys():
    print(k)
    if k == "channel_side1_ant1_tone  1":
        break

In [ ]:
dft_d = np.gradient(df['%Timestamp[s]'])
new_m_ind = [0] + list(np.where(dft_d < -750)[0][::2]) + [len(df)]
new_m_ind

In [ ]:
def cnn_dataset(df, new_m_ind, dt=10):
    max_pad = 0
    tods_ap = []
    spectrs_ap = []
    ps_xy = []

    for i in range(len(new_m_ind)-1):
        dfmini = df.loc[new_m_ind[i]+2:new_m_ind[i+1]+1]

        mint = dfmini["%Timestamp[s]"].min()
        dfmini["%Timestamp[s]"] += -mint
        widx = 0
        while True:
            tod_ap = []
            spectr_ap = []
            data = dfmini[(dfmini["%Timestamp[s]"] <= dt*(widx+1)) & (dfmini["%Timestamp[s]"] > dt*widx)].iloc[
                :, :57+11
            ]
            if len(data) == 0:
                break
            for i in range(1, 13):
                tod_ap.append(data[data["AP_index"] == i]["ToD_factor[m]"].to_numpy())
                spectr_ap.append(data[data["AP_index"] == i].iloc[:, 11:].applymap(lambda x: np.abs(complex(x.replace('i', 'j')))).to_numpy())
                if len(tod_ap[-1]) > max_pad:
                    max_pad = len(tod_ap[-1])
            tods_ap.append(tod_ap)
            spectrs_ap.append(spectr_ap)
            ps_xy.append(
                np.mean(data[[
                    'GroundTruthPositionX[m]',
                    'GroundTruthPositionY[m]'
                ]].to_numpy(),axis=0)
            )
            widx += 1
    for i in range(len(tods_ap)):
        tods_ap[i] = pad_sequences(tods_ap[i], padding="post", maxlen=max_pad)
        spectrs_ap[i] = pad_sequences(spectrs_ap[i], padding="post", maxlen=max_pad)
    tods_ap = tf.transpose(np.array(tods_ap), perm=[0,2,1]).numpy()
    spectrs_ap = tf.transpose(np.array(spectrs_ap), perm=[0,2,1,3]).numpy()
    ps_xy = np.array(ps_xy)

    spectrs_ap = np.divide(spectrs_ap - np.mean(spectrs_ap, axis=(1, 0, 2)), np.std(spectrs_ap, axis=(1, 0, 2)))
    tods_ap = np.divide(tods_ap - np.mean(tods_ap, axis=0), np.std(tods_ap, axis=0))
    ps_xy = np.divide(ps_xy - np.mean(ps_xy, axis=0),  np.std(ps_xy, axis=0))

    N_tr = tods_ap.shape[0]
    np.random.seed(5287231)
    Rarray = np.random.choice(np.arange(0, N_tr), replace=False, size=(1, N_tr)).reshape(N_tr)
    tods_ap = tods_ap[Rarray]



    ps_xy = ps_xy[Rarray]
    train_tod, test_tod, train_spectr, test_spectr, train_pxy, test_pxy = train_test_split(tods_ap, spectrs_ap, ps_xy, test_size=0.33)


    train_spectr_ap1 = train_spectr[:,:,0]
    train_spectr_ap2 = train_spectr[:,:,1]
    train_spectr_ap3 = train_spectr[:,:,2]
    train_spectr_ap4 = train_spectr[:,:,3]
    train_spectr_ap5 = train_spectr[:,:,4]
    train_spectr_ap6 = train_spectr[:,:,5]
    train_spectr_ap7 = train_spectr[:,:,6]
    train_spectr_ap8 = train_spectr[:,:,7]
    train_spectr_ap9 = train_spectr[:,:,8]
    train_spectr_ap10 = train_spectr[:,:,9]
    train_spectr_ap11 = train_spectr[:,:,10]
    train_spectr_ap12 = train_spectr[:,:,11]

    test_spectr_ap1 = test_spectr[:,:,0]
    test_spectr_ap2 = test_spectr[:,:,1]
    test_spectr_ap3 = test_spectr[:,:,2]
    test_spectr_ap4 = test_spectr[:,:,3]
    test_spectr_ap5 = test_spectr[:,:,4]
    test_spectr_ap6 = test_spectr[:,:,5]
    test_spectr_ap7 = test_spectr[:,:,6]
    test_spectr_ap8 = test_spectr[:,:,7]
    test_spectr_ap9 = test_spectr[:,:,8]
    test_spectr_ap10 = test_spectr[:,:,9]
    test_spectr_ap11 = test_spectr[:,:,10]
    test_spectr_ap12 = test_spectr[:,:,11]

    train = tf.data.Dataset.from_tensor_slices((
                tf.cast(train_tod, tf.float32),
                tf.cast(train_spectr_ap1, tf.float32),
                tf.cast(train_spectr_ap2, tf.float32),
                tf.cast(train_spectr_ap3, tf.float32),
                tf.cast(train_spectr_ap4, tf.float32),
                tf.cast(train_spectr_ap5, tf.float32),
                tf.cast(train_spectr_ap6, tf.float32),
                tf.cast(train_spectr_ap7, tf.float32),
                tf.cast(train_spectr_ap8, tf.float32),
                tf.cast(train_spectr_ap9, tf.float32),
                tf.cast(train_spectr_ap10, tf.float32),
                tf.cast(train_spectr_ap11, tf.float32),
                tf.cast(train_spectr_ap12, tf.float32),
                tf.cast(train_pxy, tf.float32),
            )).batch(1000)

    test = tf.data.Dataset.from_tensor_slices((
                tf.cast(test_tod, tf.float32),
                tf.cast(test_spectr_ap1, tf.float32),
                tf.cast(test_spectr_ap2, tf.float32),
                tf.cast(test_spectr_ap3, tf.float32),
                tf.cast(test_spectr_ap4, tf.float32),
                tf.cast(test_spectr_ap5, tf.float32),
                tf.cast(test_spectr_ap6, tf.float32),
                tf.cast(test_spectr_ap7, tf.float32),
                tf.cast(test_spectr_ap8, tf.float32),
                tf.cast(test_spectr_ap9, tf.float32),
                tf.cast(test_spectr_ap10, tf.float32),
                tf.cast(test_spectr_ap11, tf.float32),
                tf.cast(test_spectr_ap12, tf.float32),
                tf.cast(test_pxy, tf.float32),
            )).batch(1000)

    return train, test, tods_ap, spectrs_ap, ps_xy

In [ ]:
def rnn_dataset(df, new_m_ind, dt=10):
    max_pad = 0
    tods_ap = []
    ps_xy = []
    spectrs_ap = []

    for i in range(len(new_m_ind)-1):
        dfmini = df.loc[new_m_ind[i]+2:new_m_ind[i+1]+1]

        mint = dfmini["%Timestamp[s]"].min()
        dfmini["%Timestamp[s]"] += -mint
        widx = 0
        while True:
            tod_ap = []
            xy = []
            spectr_ap = []
            data = dfmini[(dfmini["%Timestamp[s]"] <= dt*(widx+1)) & (dfmini["%Timestamp[s]"] > dt*widx)].iloc[
                :, :57+11
            ]
            if len(data) == 0:
                break
            for i in range(1, 13):
                tod_ap.append(data[data["AP_index"] == i]["ToD_factor[m]"].to_numpy())
                spectr_ap.append(data[data["AP_index"] == i].iloc[:, 11:].applymap(lambda x: np.abs(complex(x.replace('i', 'j')))).to_numpy())
                xy.append(
                    data[data["AP_index"] == i][[
                        'GroundTruthPositionX[m]',
                        'GroundTruthPositionY[m]'
                    ]].to_numpy()
                )

                if len(tod_ap[-1]) > max_pad:
                    max_pad = len(tod_ap[-1])

            xy = pad_sequences(xy, padding="post")
            xy = np.true_divide(xy.sum(0),(xy!=0).sum(0))
            tods_ap.append(tod_ap)
            spectrs_ap.append(spectr_ap)
            xy = np.nan_to_num(xy)
            ps_xy.append(xy)
            widx += 1

    for i in range(len(tods_ap)):
        tods_ap[i] = pad_sequences(tods_ap[i], padding="post", maxlen=max_pad)
        spectrs_ap[i] = pad_sequences(spectrs_ap[i], padding="post", maxlen=max_pad)
        xy = ps_xy[i]
        x = np.mean(xy[:, 0])
        y = np.mean(xy[:, 1])
        ps_xy[i] = np.array([x, y])

    spectrs_ap = tf.transpose(np.array(spectrs_ap), perm=[0,2,1,3]).numpy()
    tods_ap = tf.transpose(np.array(tods_ap), perm=[0,2,1]).numpy()
    ps_xy = np.array(ps_xy)
    spectrs_ap = np.divide(spectrs_ap - np.min(spectrs_ap, axis=(1, 0, 2)), np.max(spectrs_ap, axis=(1, 0, 2)) - np.min(spectrs_ap, axis=(1, 0, 2)))
    tods_ap = np.divide(tods_ap - np.min(tods_ap, axis=(1, 0)), np.max(tods_ap, axis=(1, 0)) - np.min(tods_ap, axis=(1, 0)))
    ps_xy = np.divide(ps_xy - np.min(ps_xy, axis=(0)),  np.max(ps_xy, axis=(0)) - np.min(ps_xy, axis=(0)))

    N_tr = tods_ap.shape[0]
    np.random.seed(5287231)
    Rarray = np.random.choice(np.arange(0, N_tr), replace=False, size=(1, N_tr)).reshape(N_tr)
    tods_ap = tods_ap[Rarray]
    ps_xy = ps_xy[Rarray]
    spectrs_ap = spectrs_ap[Rarray]

    train_tod, test_tod, train_spectr, test_spectr, train_pxy, test_pxy = train_test_split(tods_ap, spectrs_ap, ps_xy, test_size=0.33)

    train_spectr_ap1 = train_spectr[:,:,0]
    train_spectr_ap2 = train_spectr[:,:,1]
    train_spectr_ap3 = train_spectr[:,:,2]
    train_spectr_ap4 = train_spectr[:,:,3]
    train_spectr_ap5 = train_spectr[:,:,4]
    train_spectr_ap6 = train_spectr[:,:,5]
    train_spectr_ap7 = train_spectr[:,:,6]
    train_spectr_ap8 = train_spectr[:,:,7]
    train_spectr_ap9 = train_spectr[:,:,8]
    train_spectr_ap10 = train_spectr[:,:,9]
    train_spectr_ap11 = train_spectr[:,:,10]
    train_spectr_ap12 = train_spectr[:,:,11]

    test_spectr_ap1 = test_spectr[:,:,0]
    test_spectr_ap2 = test_spectr[:,:,1]
    test_spectr_ap3 = test_spectr[:,:,2]
    test_spectr_ap4 = test_spectr[:,:,3]
    test_spectr_ap5 = test_spectr[:,:,4]
    test_spectr_ap6 = test_spectr[:,:,5]
    test_spectr_ap7 = test_spectr[:,:,6]
    test_spectr_ap8 = test_spectr[:,:,7]
    test_spectr_ap9 = test_spectr[:,:,8]
    test_spectr_ap10 = test_spectr[:,:,9]
    test_spectr_ap11 = test_spectr[:,:,10]
    test_spectr_ap12 = test_spectr[:,:,11]

    train = tf.data.Dataset.from_tensor_slices((
                tf.cast(train_tod, tf.float32),
                tf.cast(train_spectr_ap1, tf.float32),
                tf.cast(train_spectr_ap2, tf.float32),
                tf.cast(train_spectr_ap3, tf.float32),
                tf.cast(train_spectr_ap4, tf.float32),
                tf.cast(train_spectr_ap5, tf.float32),
                tf.cast(train_spectr_ap6, tf.float32),
                tf.cast(train_spectr_ap7, tf.float32),
                tf.cast(train_spectr_ap8, tf.float32),
                tf.cast(train_spectr_ap9, tf.float32),
                tf.cast(train_spectr_ap10, tf.float32),
                tf.cast(train_spectr_ap11, tf.float32),
                tf.cast(train_spectr_ap12, tf.float32),
                tf.cast(train_pxy, tf.float32),
            )).batch(1000)

    test = tf.data.Dataset.from_tensor_slices((
                tf.cast(test_tod, tf.float32),
                tf.cast(test_spectr_ap1, tf.float32),
                tf.cast(test_spectr_ap2, tf.float32),
                tf.cast(test_spectr_ap3, tf.float32),
                tf.cast(test_spectr_ap4, tf.float32),
                tf.cast(test_spectr_ap5, tf.float32),
                tf.cast(test_spectr_ap6, tf.float32),
                tf.cast(test_spectr_ap7, tf.float32),
                tf.cast(test_spectr_ap8, tf.float32),
                tf.cast(test_spectr_ap9, tf.float32),
                tf.cast(test_spectr_ap10, tf.float32),
                tf.cast(test_spectr_ap11, tf.float32),
                tf.cast(test_spectr_ap12, tf.float32),
                tf.cast(test_pxy, tf.float32),
            )).batch(1000)

    return train, test, tods_ap, spectrs_ap, ps_xy

In [ ]:
# from bayes opt
dt = 7.632100041832579
# simple
dt = 4
train, test, _, _, _ = rnn_dataset(df, new_m_ind, dt)

In [ ]:
class MyCNN(tf.keras.layers.Layer):
    def __init__(self, N, name="cnn"):
        super(MyCNN, self).__init__(name=name)

        self.cnn1 = Conv1D(N,3,activation=None,
                           bias_initializer=tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5),
                           name='cnn1')

        self.cnn2 = Conv1D(N,3,activation=None,
                           bias_initializer=tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5),
                           padding='same', name='cnn2')

        self.sifmoid = tf.keras.activations.selu

    def call(self, inpt):
        x = self.cnn1(inpt)
        x1 = self.sifmoid(x)

        x = self.cnn2(x1)
        x = x + x1
        y = self.sifmoid(x)

        return y

class RTT_CNN1D(Model, BaseNeuralNetwork):

    def __init__(self, early_stop_vars=None):
        Model.__init__(self, name="rtt_cnn")
        status = [
            [0],
            [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0],
            [1,2]
        ]
        BaseNeuralNetwork.__init__(self, status, early_stop_vars, None, "Adam", 1e-4)
        self.score_mtr = MetricBase(self,
            [mCosS()],
            status,
            [0]
        )

        self.cost_mtr = MetricBase(self,
            [mMSE()],
            status,
            [0],
            1
        )

        self.cost_loss = LossBase(self,
          [lMSE()],
          status,
          [0]
        )


        self.cnn1 = MyCNN(128, 'cnn1')

        cnnN = 64
        self.cnn_ap1 = MyCNN(cnnN, 'cnn_ap1')
        self.cnn_ap2 = MyCNN(cnnN, 'cnn_ap2')
        self.cnn_ap3 = MyCNN(cnnN, 'cnn_ap3')
        self.cnn_ap4 = MyCNN(cnnN, 'cnn_ap4')
        self.cnn_ap5 = MyCNN(cnnN, 'cnn_ap5')
        self.cnn_ap6 = MyCNN(cnnN, 'cnn_ap6')
        self.cnn_ap7 = MyCNN(cnnN, 'cnn_ap7')
        self.cnn_ap8 = MyCNN(cnnN, 'cnn_ap8')
        self.cnn_ap9 = MyCNN(cnnN, 'cnn_ap9')
        self.cnn_ap10 = MyCNN(cnnN, 'cnn_ap10')
        self.cnn_ap11 = MyCNN(cnnN, 'cnn_ap11')
        self.cnn_ap12 = MyCNN(cnnN, 'cnn_ap12')

        self.concat = tf.keras.layers.Concatenate()

        self.flatten = Flatten()

        self.denseout = Dense(2,activation='sigmoid',name="denseout")

    def call(self, inputs):

        x = self.cnn1(inputs[0])

        x1 = self.cnn_ap1(inputs[1])
        x2 = self.cnn_ap1(inputs[2])
        x3 = self.cnn_ap1(inputs[3])
        x4 = self.cnn_ap1(inputs[4])
        x5 = self.cnn_ap1(inputs[5])
        x6 = self.cnn_ap1(inputs[6])
        x7 = self.cnn_ap1(inputs[7])
        x8 = self.cnn_ap1(inputs[8])
        x9 = self.cnn_ap1(inputs[9])
        x10 = self.cnn_ap1(inputs[10])
        x11 = self.cnn_ap1(inputs[11])
        x12 = self.cnn_ap1(inputs[12])

        xap = x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10 + x11 + x12
        x = self.concat([x , xap])

        x = self.flatten(x)

        y = self.denseout(x)
        return tuple((y,))

In [ ]:
class Ap_LSTM(tf.keras.layers.Layer):
    def __init__(self, lstmN, return_s=False, bid=False):
        def set_bid(lstmN, return_s, bid, name):
            if bid:
                return Bidirectional(
                    LSTM(lstmN, return_sequences=return_s,
                         bias_initializer=tf.keras.initializers.RandomUniform(minval=-2, maxval=2),
                    ),
                    name=name
                )
            else:
                return LSTM(lstmN, return_sequences=return_s,name=name,
                            bias_initializer=tf.keras.initializers.RandomUniform(minval=-2, maxval=2)
                       )

        super(Ap_LSTM, self).__init__(name='Ap_LSTM')
        for i in range(1, lstmN+1):
            setattr(
                self,
                'lstm_ap{}'.format(str(i)),
                set_bid(lstmN, return_s, bid,'lstm_ap{}'.format(str(i)))
            )

    def call(self, inputs, comp_sum=False):
        y = []
        for i, inpt in enumerate(inputs):
            y.append(
                getattr(self, 'lstm_ap{}'.format(str(i+1)))(inpt)
            )

        if comp_sum:
            y = sum(y)

        return y


class RTT_RNN(Model, BaseNeuralNetwork):
    def __init__(self, early_stop_vars=None):
        Model.__init__(self, name="rtt_rnn")
        status = [
            [0],
            [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0],
            [1,2]
        ]
        BaseNeuralNetwork.__init__(self, status, early_stop_vars, None, "Adam", 1e-4)
        self.score_mtr = MetricBase(self,
            [mCosS()],
            status,
            [0]
        )

        self.cost_mtr = MetricBase(self,
            [mMSE()],
            status,
            [0],
            1
        )

        self.cost_loss = LossBase(self,
          [lMSE()],
          status,
          [0]
        )
        self.lstm1 =  LSTM(32, return_sequences=True,name="lstm1")
        self.bidirectional1 = Bidirectional(self.lstm1,name="bidirectional1")

        self.lstm2 =  LSTM(32, return_sequences=False,name="lstm1")
        self.bidirectional2 = Bidirectional(self.lstm2,name="bidirectional2")

        lstmN = 32
        self.ap_lstm1 = Ap_LSTM(lstmN,True, bid=True)
        self.ap_lstm2 = Ap_LSTM(lstmN,False, bid=True)

        self.concat = tf.keras.layers.Concatenate()
        self.dense = Dense(256,activation='tanh',name="dense")

        self.denseout = Dense(2,activation='sigmoid',name="denseout")

    def call(self, inputs):

        x = self.bidirectional1(inputs[0])

        x = self.bidirectional2(x)

        xap = self.ap_lstm1(inputs[1:12], False)
        xap = self.ap_lstm2(xap, True)

        x = self.concat([x , xap])
        x = self.dense(x)

        y = self.denseout(x)
        return tuple((y,))


In [ ]:
rtt_cnn = RTT_CNN1D()

In [ ]:
rtt_cnn.set_lr_rate(1e-4)
rtt_cnn.set_clip_norm(0)
rtt_cnn.train(train, 1000, test, history_learning_process=True)

In [ ]:
es = {'es_strategy':'patience', 'es_metric':'val_cost', 'es_min_delta': 1e-7, 'es_patience': 40 }
rtt_rnn = RTT_RNN(early_stop_vars=es)

In [ ]:
rtt_rnn.set_lr_rate(1e-4)
rtt_rnn.set_early_stop({'es_strategy':'patience', 'es_metric':'val_cost', 'es_min_delta': 1e-7, 'es_patience': 40 })
rtt_rnn.set_clip_norm(0)
rtt_rnn.train(train, 600, test, history_learning_process=True)

In [ ]:
def plot_psxy(model, plotd='train', kfirst=10, save_obj=None):

    if plotd == 'train':
        for t in train:
            pxy_t = t[-1]
    elif plotd == 'test':
        for t in test:
            pxy_t = t[-1]

    pxy_t_ = model(t)[0]
    k = tf.reduce_sum(tf.subtract(pxy_t, pxy_t_)**2,axis=1)
    k = tf.argsort(k)
    k = k[:kfirst].numpy().tolist()
    pxy_t = pxy_t.numpy()[k]
    pxy_t_ = pxy_t_.numpy()[k]

    plt.figure(figsize=(16, 8))
    plt.scatter(pxy_t[:, 0], pxy_t[:, 1], marker='^', label='actual')
    plt.scatter(pxy_t_[:, 0], pxy_t_[:, 1], marker='*', label='predict')
    xtics = [-45.0, -35.0, -25.0, -15.0, -5.0, 5.0]
    ytics = [-5, 0 , 5, 10, 15, 20]
    plt.xticks(np.linspace(0,1,6), xtics, fontsize=16)
    plt.yticks(np.linspace(0,1,6), ytics, fontsize=16)
    plt.legend(fontsize=20)

    if save_obj is not None:
        plt.savefig('{}/DataFigures/{}-{}.svg'.format(*save_obj))
    else:
        plt.xticks(np.linspace(0,1,6), xtics, fontsize=16, color='red')
        plt.yticks(np.linspace(0,1,6), ytics, fontsize=16, color='red')
        plt.show()

In [ ]:
#train
save_obj = ['.','outpout','train-top-20']
plot_psxy(rtt_rnn, kfirst=20, save_obj=None)

In [ ]:
#test
save_obj = ['.','outpout','test-top-20']
plot_psxy(rtt_rnn, kfirst=20, plotd='test', save_obj=None)

In [ ]:
save_obj = ['.','.','rtt-wifi','cost-score','']
history_figure(rtt_rnn.history,save_obj=None)